# Cars: Grouping and Aggregation

## Imports

In [1]:
import pandas as pd
import numpy as np
from altair import load_dataset

## Dataset

In [28]:
cars = load_dataset('cars')

In [29]:
cars.head()

,Acceleration,Cylinders,Displacement,Horsepower,Miles_per_Gallon,Name,Origin,Weight_in_lbs,Year
0,12.0,8,307.0,130.0,18.0,chevrolet chevelle malibu,USA,3504,1970-01-01
1,11.5,8,350.0,165.0,15.0,buick skylark 320,USA,3693,1970-01-01
2,11.0,8,318.0,150.0,18.0,plymouth satellite,USA,3436,1970-01-01
3,12.0,8,304.0,150.0,16.0,amc rebel sst,USA,3433,1970-01-01
4,10.5,8,302.0,140.0,17.0,ford torino,USA,3449,1970-01-01


In [30]:
len(cars)

406

## Grouped mean

Compute the average of `Acceleration`, `Displacement` and `Horsepower`, grouped by `Origin`:

In [31]:
# YOUR CODE HERE
g1 = pd.DataFrame(cars.groupby('Origin')['Acceleration', 'Displacement', 'Horsepower'].mean())
g1

,Acceleration,Displacement,Horsepower
Origin,,,
Europe,16.821918,109.465753,81.000000
Japan,16.172152,102.708861,79.835443
USA,14.942520,247.935039,119.900000


In [32]:
assert list(g1.columns)==['Acceleration', 'Displacement', 'Horsepower']
assert list(g1.index)==['Europe', 'Japan', 'USA']

Find the maximum `Acceleration` and `Displacement` grouped by `Origin` and `Cylinders`:

In [33]:
# YOUR CODE HERE
g2 = pd.DataFrame(cars.groupby(['Origin', 'Cylinders'])['Acceleration', 'Displacement'].max())
g2

Acceleration  Displacement
Origin Cylinders                            
Europe 4                  24.8         146.0
       5                  20.1         183.0
       6                  19.6         168.0
Japan  3                  13.5          80.0
       4                  21.0         144.0
       6                  15.5         168.0
USA    4                  22.2         156.0
       6                  21.0         262.0
       8                  22.2         455.0

In [34]:
assert list(g2.columns)==['Acceleration', 'Displacement']
assert [list(i) for i in list(g2.index.levels)]==[['Europe', 'Japan', 'USA'], [3, 4, 5, 6, 8]]

## Grouped counts

Compute the number of cars, grouped by `Year` and `Cylinders` and unstack the result:

In [35]:
# YOUR CODE HERE
g3 = cars.groupby(['Year','Cylinders'])['Cylinders'].sum().unstack()
g3

Cylinders,3,4,5,6,8
Year,,,,,
1970-01-01,NaN,32.0,NaN,24.0,184.0
1971-01-01,NaN,56.0,NaN,48.0,56.0
1972-01-01,3.0,56.0,NaN,NaN,104.0
1973-01-01,3.0,44.0,NaN,48.0,160.0
1974-01-01,NaN,60.0,NaN,42.0,40.0
1975-01-01,NaN,48.0,NaN,72.0,48.0
1976-01-01,NaN,60.0,NaN,60.0,72.0
1977-01-01,3.0,56.0,NaN,30.0,64.0
1978-01-01,NaN,68.0,5.0,72.0,48.0


In [36]:
assert list(g3.columns)==[3, 4, 5, 6, 8]
assert list(g3.index)==['1970-01-01', '1971-01-01', '1972-01-01', '1973-01-01',
                        '1974-01-01', '1975-01-01', '1976-01-01', '1977-01-01', 
                        '1978-01-01', '1979-01-01', '1980-01-01', '1982-01-01']

## Multiple aggregations for different columns

Compute two aggregate quantities with the following names and values:

* `min_mpg` as the minimum `Miles_per_Gallon`.
* `max_mpg` as the maximum `Miles_per_Gallon`.
 
Group by `Origin`:

In [40]:
# YOUR CODE HERE
g4 = cars.groupby('Origin')['Miles_per_Gallon'].aggregate({'min_mpg': 'min',
                                                           'max_mpg': 'max'
                                                    })
g4 = g4[['max_mpg', 'min_mpg']]
g4

,max_mpg,min_mpg
Origin,,
Europe,44.3,16.2
Japan,46.6,18.0
USA,39.0,9.0


In [41]:
assert list(g4.columns)==['max_mpg', 'min_mpg']
assert list(g4.index)==['Europe', 'Japan', 'USA']

## Custom aggregation function

Compute the range of cylinders (`range` = max - min) grouped by `Origin`:

In [19]:
# YOUR CODE HERE
g5 = cars.groupby('Origin')['Cylinders'].aggregate({'range': lambda x: x.max() - x.min()})
g5

,range
Origin,
Europe,2
Japan,3
USA,4


In [20]:
assert list(g5.columns)==['range']
assert list(g5.index)==['Europe', 'Japan', 'USA']

## Group filtering

Compute the average acceleration, grouped by `Year` and `Origin`, only including groups with a max number of Cylinders less than 6. Unstack the `Origin` level of the resulting hierarchical row index:

In [26]:
# YOUR CODE HERE
def filter_cyl(x):
    return x['Cylinders'].max() < 6

test = cars.groupby(['Year','Origin']).filter(filter_cyl)
g6 = pd.DataFrame(test.groupby(['Year','Origin'])['Acceleration'].mean()).unstack()
g6

Acceleration           
Origin           Europe      Japan
Year                              
1970-01-01    16.666667  14.750000
1971-01-01    17.400000  16.375000
1972-01-01    18.700000  15.400000
1973-01-01    16.428571        NaN
1974-01-01    15.333333  17.666667
1975-01-01    15.083333  16.000000
1977-01-01    15.000000        NaN
1978-01-01          NaN  16.200000
1979-01-01    18.400000  17.200000
1980-01-01    18.366667        NaN

In [27]:
assert [list(i) for i in list(g6.columns.levels)]==[['Acceleration'], ['Europe', 'Japan']]
assert g6.index.name=='Year'
assert len(g6)==10

## Grouped z-scores

Here is the average `Miles_per_Gallon`, grouped by `Year`:

In [42]:
cars.groupby(['Year'])['Miles_per_Gallon'].mean()

Year
1970-01-01    17.689655
1971-01-01    21.250000
1972-01-01    18.714286
1973-01-01    17.100000
1974-01-01    22.703704
1975-01-01    20.266667
1976-01-01    21.573529
1977-01-01    23.375000
1978-01-01    24.061111
1979-01-01    25.093103
1980-01-01    33.696552
1982-01-01    31.045000
Name: Miles_per_Gallon, dtype: float64

Replace the `Miles_per_Gallon` values by the [z-score](https://en.wikipedia.org/wiki/Standard_score) of that value relative to its group for each `Year`:

In [43]:
# YOUR CODE HERE
def z_score(x):
    x['Miles_per_Gallon'] = (x['Miles_per_Gallon'] - x['Miles_per_Gallon'].mean())/ x['Miles_per_Gallon'].std()
    return x

d7 = pd.DataFrame(cars.groupby(['Year']).apply(z_score).dropna())
d7

,Acceleration,Cylinders,Displacement,Horsepower,Miles_per_Gallon,Name,Origin,Weight_in_lbs,Year
0,12.0,8,307.0,130.0,0.058125,chevrolet chevelle malibu,USA,3504,1970-01-01
1,11.5,8,350.0,165.0,-0.503753,buick skylark 320,USA,3693,1970-01-01
2,11.0,8,318.0,150.0,0.058125,plymouth satellite,USA,3436,1970-01-01
3,12.0,8,304.0,150.0,-0.316460,amc rebel sst,USA,3433,1970-01-01
4,10.5,8,302.0,140.0,-0.129168,ford torino,USA,3449,1970-01-01
5,10.0,8,429.0,198.0,-0.503753,ford galaxie 500,USA,4341,1970-01-01
6,9.0,8,454.0,220.0,-0.691046,chevrolet impala,USA,4354,1970-01-01
7,8.5,8,440.0,215.0,-0.691046,plymouth fury iii,USA,4312,1970-01-01
8,10.0,8,455.0,225.0,-0.691046,pontiac catalina,USA,4425,1970-01-01
9,8.5,8,390.0,190.0,-0.503753,amc ambassador dpl,USA,3850,1970-01-01


Show that the average z-scores for `Miles_per_Gallon` grouped by year are all zero:

In [44]:
# YOUR CODE HERE
d7.groupby(['Year']).mean()

,Acceleration,Cylinders,Displacement,Horsepower,Miles_per_Gallon,Weight_in_lbs
Year,,,,,,
1970-01-01,12.948276,6.758621,281.413793,147.827586,-7.656711e-17,3372.793103
1971-01-01,15.000000,5.629630,213.888889,107.037037,-2.106949e-02,3030.592593
1972-01-01,15.125000,5.821429,218.375000,120.178571,-2.299748e-16,3237.714286
1973-01-01,14.312500,6.375000,256.875000,130.475000,-3.053113e-16,3419.025000
1974-01-01,16.173077,5.230769,170.653846,94.230769,1.020669e-02,2878.038462
1975-01-01,16.050000,5.600000,205.533333,101.066667,2.960595e-16,3176.800000
1976-01-01,15.941176,5.647059,197.794118,101.117647,-5.224579e-17,3078.735294
1977-01-01,15.435714,5.464286,191.392857,105.071429,1.189525e-17,2997.357143
1978-01-01,15.805556,5.361111,177.805556,99.694444,-3.484867e-16,2861.805556


Show that the standard deviation of the z-scores for `Miles_per_Gallon` grouped by year are all 1.0:

In [45]:
# YOUR CODE HERE
d7.groupby(['Year']).std()

,Acceleration,Cylinders,Displacement,Horsepower,Miles_per_Gallon,Weight_in_lbs
Year,,,,,,
1970-01-01,3.330982,1.724926,124.421380,53.734844,1.000000,852.868663
1971-01-01,2.605468,1.667521,115.152185,38.566109,1.012696,1065.314842
1972-01-01,2.850032,2.073708,123.781964,41.121368,1.000000,974.520960
1973-01-01,2.754222,1.807215,121.722085,46.412304,1.000000,974.809133
1974-01-01,1.714306,1.607674,94.259193,29.686775,1.018369,968.108402
1975-01-01,2.471737,1.522249,87.669730,26.577062,1.000000,765.179781
1976-01-01,2.801419,1.667558,94.422256,32.430592,1.000000,821.371481
1977-01-01,2.273391,1.815206,107.813742,36.095479,1.000000,912.825902
1978-01-01,2.129915,1.495761,76.012713,28.436214,1.000000,626.023907


## Grouped missing value replacement

## Introduce missing values

Let's introduce some missing values into the `Cylinders` column:

In [46]:
cars['Cylinders'] = np.where(np.random.rand(len(cars)) > 0.8, np.nan, cars['Cylinders'])
cars.Cylinders

0      NaN
1      NaN
2      8.0
3      8.0
4      8.0
5      NaN
6      8.0
7      8.0
8      8.0
9      NaN
10     4.0
11     8.0
12     8.0
13     8.0
14     8.0
15     8.0
16     NaN
17     8.0
18     NaN
19     NaN
20     4.0
21     NaN
22     6.0
23     6.0
24     4.0
25     4.0
26     4.0
27     NaN
28     4.0
29     4.0
      ... 
376    4.0
377    4.0
378    4.0
379    4.0
380    4.0
381    4.0
382    4.0
383    4.0
384    4.0
385    4.0
386    4.0
387    4.0
388    4.0
389    4.0
390    4.0
391    4.0
392    4.0
393    NaN
394    6.0
395    6.0
396    4.0
397    NaN
398    4.0
399    NaN
400    4.0
401    4.0
402    NaN
403    4.0
404    4.0
405    4.0
Name: Cylinders, dtype: float64

In [47]:
cars.head()

,Acceleration,Cylinders,Displacement,Horsepower,Miles_per_Gallon,Name,Origin,Weight_in_lbs,Year
0,12.0,NaN,307.0,130.0,18.0,chevrolet chevelle malibu,USA,3504,1970-01-01
1,11.5,NaN,350.0,165.0,15.0,buick skylark 320,USA,3693,1970-01-01
2,11.0,8.0,318.0,150.0,18.0,plymouth satellite,USA,3436,1970-01-01
3,12.0,8.0,304.0,150.0,16.0,amc rebel sst,USA,3433,1970-01-01
4,10.5,8.0,302.0,140.0,17.0,ford torino,USA,3449,1970-01-01


Here are the average number of Cylinders, grouped by `Year`:

In [48]:
cars.groupby(['Year'])['Cylinders'].mean()

Year
1970-01-01    6.640000
1971-01-01    5.391304
1972-01-01    5.352941
1973-01-01    6.346154
1974-01-01    5.333333
1975-01-01    5.750000
1976-01-01    5.733333
1977-01-01    5.285714
1978-01-01    5.137931
1979-01-01    5.730769
1980-01-01    4.181818
1982-01-01    4.307692
Name: Cylinders, dtype: float64

Replace the missing values in `Cylinders` by the group average (grouped by `Year`):

In [49]:
# YOUR CODE HERE
cars['Cylinders'] = cars.groupby(['Year'])['Cylinders'].transform(lambda x: x.fillna(x.mean()))

#df.groupby("name").transform(lambda x: x.fillna(x.mean()))

By replacing missing values with group averages, the grouped averages remain unchanged:

In [50]:
# YOUR CODE HERE
cars.groupby(['Year'])['Cylinders'].mean()

Year
1970-01-01    6.640000
1971-01-01    5.391304
1972-01-01    5.352941
1973-01-01    6.346154
1974-01-01    5.333333
1975-01-01    5.750000
1976-01-01    5.733333
1977-01-01    5.285714
1978-01-01    5.137931
1979-01-01    5.730769
1980-01-01    4.181818
1982-01-01    4.307692
Name: Cylinders, dtype: float64